In [1]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

from scipy import stats
from Bio import SeqIO

import collections
from collections import OrderedDict
from collections import Counter

from skbio.diversity import alpha_diversity
from skbio.diversity import beta_diversity
from io import StringIO
from skbio import TreeNode

from skbio.stats.distance import mantel
from skbio.stats.distance import anosim
from skbio.stats.distance import permanova
from skbio.stats.ordination import pcoa

In [2]:
dfuc95 = pd.read_csv('Data/result_uc95_list', header=None, sep="|", usecols=[0,2], index_col=1, names=["clst", "accid"])
dfuc95['clst']=dfuc95['clst'].map(lambda x: 'OTU'+str(x))
print(len(dfuc95))

1198


In [3]:
Italic =   ['$\it{In}$ $\it{situ}$ total RNA 1', '$\it{In}$ $\it{situ}$ total RNA 2', '$\it{In}$ $\it{situ}$ total RNA 3', '$\it{In}$ $\it{situ}$ total RNA 4', '$\it{In}$ $\it{situ}$ total RNA 5',
       '$\it{In}$ $\it{situ}$ total RNA 6', 'Onboard total RNA 1', 'Onboard total RNA 2', 'Onboard total RNA 3',
       'Onboard total RNA 4', 'Onboard total RNA 5', 'Onboard total RNA 6', 'Onboard total RNA 7']

dfcount = pd.read_csv('Data/count', header=0, index_col=0, sep='\t')
dfcount.head()

dfcount_clst=pd.concat([dfuc95, dfcount], axis=1, sort=False)

otutable=dfcount_clst.groupby('clst').sum()
otutable=otutable.astype(int)
otutable.columns=Italic

otutable.drop(otutable.columns[[0]], axis=1, inplace=True)

In [4]:
Italic =   ['$\it{In}$ $\it{situ}$ total RNA 2', '$\it{In}$ $\it{situ}$ total RNA 3', '$\it{In}$ $\it{situ}$ total RNA 4', '$\it{In}$ $\it{situ}$ total RNA 5',
       '$\it{In}$ $\it{situ}$ total RNA 6', 'Onboard total RNA 1', 'Onboard total RNA 2', 'Onboard total RNA 3',
       'Onboard total RNA 4', 'Onboard total RNA 5', 'Onboard total RNA 6', 'Onboard total RNA 7']

data = [list(otutable[Italic[0]]), list(otutable[Italic[1]]), list(otutable[Italic[2]]), list(otutable[Italic[3]]), list(otutable[Italic[4]]),
           list(otutable[Italic[5]]), list(otutable[Italic[6]]), list(otutable[Italic[7]]), list(otutable[Italic[8]]), list(otutable[Italic[9]]),
           list(otutable[Italic[10]]), list(otutable[Italic[11]])]

# Alpha beta diversity

In [5]:
from scipy import stats

def MannWhitneyU(df):
    A=list(df[df['subject']=='insitu']['data'])
    B=list(df[df['subject']=='onboard']['data'])
    print(stats.mannwhitneyu(A, B, alternative='two-sided'))

In [6]:
obs_otus=pd.DataFrame()
shannon=pd.DataFrame()
faith_pd=pd.DataFrame()

subjects=['insitu','insitu','insitu','insitu','insitu','onboard','onboard','onboard','onboard','onboard','onboard','onboard']

# observed OTUs
obs_otus['data']=alpha_diversity('observed_otus', data, Italic)
obs_otus['subject']=subjects
obs_otus['metrics'] = 'Observed OTUs'
MannWhitneyU(obs_otus)

# shannon
shannon['data']=alpha_diversity('shannon', data, Italic)
shannon['subject']=subjects
shannon['metrics'] = 'Shannon diversity'
MannWhitneyU(shannon)

# Faith’s PD
otu_ids=list(otutable.index)

with open('/nfs_share/motoki/metatra/fasttree/fasttree_GTRCAT_mafft') as f:
    s = f.read()
tree = TreeNode.read(StringIO(s)).root_at_midpoint()

faith_pd['data']=alpha_diversity('faith_pd', data, ids=Italic, otu_ids=otu_ids, tree=tree)
faith_pd['subject']=subjects
faith_pd['metrics'] = "Faith's PD"
MannWhitneyU(faith_pd)

MannwhitneyuResult(statistic=24.0, pvalue=0.3290121841305219)
MannwhitneyuResult(statistic=35.0, pvalue=0.005766120914237261)
MannwhitneyuResult(statistic=26.0, pvalue=0.19387643074206262)


# Beta diversity

In [7]:
sample_md = pd.DataFrame(index=Italic)
sample_md['ids']=Italic
sample_md['subject']=subjects
#sample_md

In [8]:
# weighted UniFrac distances
wu_dm = beta_diversity("weighted_unifrac", data, Italic, tree=tree,
                       otu_ids=otu_ids)

# PERMANOVA 
results = permanova(wu_dm, sample_md, column='subject', permutations=999)
print('PERMANOVA:', results['test statistic']) #0.7632275132275134
print(results['p-value'])
print(results['p-value'] < 0.1)

PERMANOVA: 30.09906597294752
0.002
True
